In [1]:
import torch

* Các hàm (đặc biệt trong neural network) dù phức tạp nhưng cũng chỉ là hợp của các phép toán/hàm cơ bản (cộng, trừ, nhân, chia, sin, cos, exp, log, v.v.). Các phép toán/hàm cơ bản ấy đều có công thức đạo hàm $\rightarrow$ áp dụng chain rule là có thể tính được đạo hàm của bất kì hợp các hàm cơ bản.

* **Computational graph**: biểu diễn hàm hợp phức tạp ở trên bằng đồ thị. Các node là các biến/tensor, các mũi tên biểu diễn các phép toán/hàm cơ bản.

* **Automatic differentiation (autodiff)**: kĩ thuật tính đạo hàm bằng chain rule dựa vào computational graph. Có 2 kĩ thuật: forward mode autodiff và reverse mode autodiff. Ở đây ta chỉ quan tâm reverse mode autodiff (backpropagation trong deep learning).

Ví dụ: 
$$y = (x - 1)^2 + (x + 1)^2$$
![autodiff](resources/autodiff.png)
\begin{align}
a &= x - 1 \\
b &= x + 1 \\
c &= a^2 \\
d &= b^2 \\
y &= c + d
\end{align}

Tính $\dfrac{\partial y}{\partial x}$ theo chain rule
\begin{align}
\dfrac{\partial y}{\partial x} &= \text{tổng các đường từ x $\rightarrow$ y} \\
&= \underbrace{\dfrac{\partial y}{\partial c}\dfrac{\partial c}{\partial a}}_{\dfrac{\partial y}{\partial a}}\dfrac{\partial a}{\partial x} + \underbrace{\dfrac{\partial y}{\partial d}\dfrac{\partial d}{\partial b}}_{\dfrac{\partial y}{\partial b}}\dfrac{\partial b}{\partial x}
\end{align}

Kí hiệu $\dfrac{\partial y}{\partial x}$ là $\overline{x}$ (tương tự $\dfrac{\partial y}{\partial a}$ là $\overline{a}$, $\dfrac{\partial y}{\partial b}$ là $\overline{b}, \ldots$) thì

$$\overline{x} = \overline{a} \cdot \dfrac{\partial a}{\partial x} + \overline{b} \cdot \dfrac{\partial b}{\partial x}$$
$\dfrac{\partial a}{\partial x}$ và $\dfrac{\partial b}{\partial x}$ đều là đạo hàm của các phép toán cơ bản nên tính được đơn giản. Vì vậy nếu biết $\overline{a}$ và $\overline{b}$ (để ý $a$ và $b$ đều là child node của $x$) thì sẽ tính được $\overline{x}$. $\overline{a}$ tính được nếu biết $\overline{c}$, $\overline{b}$ tính được nếu biết $\overline{d}$. Mà $\overline{c}$ và $\overline{d}$ cũng tính được đơn giản vì đều là đạo hàm của phép toán cơ bản $\rightarrow$ ý tưởng reverse mode autodiff.
![autodiff2](resources/autodiff2.png)
Chi tiết các bước tính toán:
1. Tính $\overline{c}$ và $\overline{d}$
\begin{align}
\overline{c} &= \dfrac{\partial y}{\partial c} = \dfrac{\partial}{\partial c} (c + d) = 1 \\
\overline{d} &= \dfrac{\partial y}{\partial d} = \dfrac{\partial}{\partial d} (c + d) = 1
\end{align}
2. Tính $\overline{a}$ và $\overline{b}$
\begin{align}
\overline{a} &= \overline{c} \cdot \dfrac{\partial c}{\partial a} = 1 \cdot \dfrac{\partial}{\partial a} a^2 = 2a \\
\overline{b} &= \overline{d} \cdot \dfrac{\partial d}{\partial b} = 1 \cdot \dfrac{\partial}{\partial b} b^2 = 2b
\end{align}
3. Tính $\overline{x}$
\begin{align}
\overline{x} &= \overline{a} \cdot \dfrac{\partial a}{\partial x} + \overline{b} \cdot \dfrac{\partial b}{\partial x} \\
&= 2a \cdot 1 + 2b \cdot 1 = 4x
\end{align}

Tính đạo hàm với `torch.autograd`

In [2]:
x = torch.tensor(4.5, requires_grad=True) # set requires_grad=True để mọi operation trên với tensor này đều được record
a = x - 1
b = x + 1
c = a**2
d = b**2
y = c + d
print(f'x = {x}')
print(f'a = x - 1 = {a}')
print(f'b = x + 1 = {b}')
print(f'c = a^2 = {c}')
print(f'd = b^2 = {d}')
print(f'y = c + d = {y}')

x = 4.5
a = x - 1 = 3.5
b = x + 1 = 5.5
c = a^2 = 12.25
d = b^2 = 30.25
y = c + d = 42.5


Gọi `y.backward()` để tính đạo hàm của `y` theo "graph leaves" (ở đây là biến `x`)

(**Note**: thông thường node `x` trong ví dụ này hay được gọi là root node, còn `y` mới gọi là leaf node. Có lẽ tutorial của pytorch cố tình đảo ngược tên gọi như thế vì trong context deeplearning thường thì chỉ có 1 output `y` trong khi có thể có nhiều biến input `x` $\rightarrow$ gọi output là root và gọi các biến input là các leaves nghe trực quan hơn)

In [3]:
y.backward() # Computes the gradient of current tensor w.r.t. graph leaves

Đạo hàm của `y` theo `x` được lưu ở attribute `x.grad`

In [4]:
print(f'dy/dx = {x.grad}')

dy/dx = 18.0


* Ví dụ 2: Tính đạo hàm của hàm sigmoid
$$y = \sigma(x) = \dfrac{1}{1 + e^{-x}}$$
![autodiff3](resources/autodiff3.png)

In [5]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))
x = torch.tensor(1.0, requires_grad=True)
y = sigmoid(x)
print(f'x = {x}')
print(f'y = sigmoid({x}) = {y}')

x = 1.0
y = sigmoid(1.0) = 0.7310585975646973


Có thể kiểm chứng đạo hàm của hàm sigmoid $\sigma(x)$ là $\sigma(x)\cdot(1 - \sigma(x))$

In [6]:
y.backward()
print(f'dy/dx = {x.grad}')
print(f'y.(1 - y) = {y*(1 - y)}')

dy/dx = 0.1966119408607483
y.(1 - y) = 0.1966119259595871


* Ví dụ 3: nhiều biến đầu vào
\begin{align}
a &= x + t \\
b &= x - t \\
y &= a \cdot b = (x + t) \cdot (x - t)
\end{align}
Tính $\overline{x} = \dfrac{\partial y}{\partial x}$ và $\overline{t} = \dfrac{\partial y}{\partial t}$ 
![autodiff4](resources/autodiff4.png)

In [7]:
x = torch.tensor(4.5, requires_grad=True)
t = torch.tensor(2.5, requires_grad=True)
a = x + t
b = x - t
y = a*b
print(f'x = {x}')
print(f't = {t}')
print(f'a = x + t = {a}')
print(f'b = x - t = {b}')
print(f'y = a*b = {y}')

x = 4.5
t = 2.5
a = x + t = 7.0
b = x - t = 2.0
y = a*b = 14.0


In [8]:
y.backward()
print(f'dy/dx = {x.grad}')
print(f'dy/dt = {t.grad}')

dy/dx = 9.0
dy/dt = -5.0


## Khi các node là các tensor
$y=f(x)$
* Cả $x$ và $y$ đều là scalar $\rightarrow$ đạo hàm cũng là scalar
* $y$ scalar, $x$ vector $n$ chiều $\rightarrow$ đạo hàm là vector gradient $n$ chiều
* $y$ vector $m$ chiều, $x$ vector $n$ chiều $\rightarrow$ đạo hàm là ma trận jacobian $m \times n$

Ví dụ: 
\begin{align}
\mathbf{x} &= 
\begin{bmatrix}
x_1 & x_2 & x_3
\end{bmatrix} \\
\mathbf{y} &= \mathbf{x}^2 = 
\begin{bmatrix}
x_1^2 & x_2^2 & x_3^2
\end{bmatrix} \\
z &= sum(\mathbf{y}) = y_1 + y_2 + y_3 = x_1^2 + x_2^2 + x_3^2
\end{align}
![autodiff5](resources/autodiff5.png)

* $\overline{\mathbf{y}} = \dfrac{\partial z}{\partial \mathbf{y}} = 
\begin{bmatrix} \overline{y}_1 & \overline{y}_2 & \overline{y}_3 \end{bmatrix} =
\begin{bmatrix} \dfrac{\partial z}{\partial y_1} & \dfrac{\partial z}{\partial y_2} & \dfrac{\partial z}{\partial y_3} \end{bmatrix} = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix}$
* $\overline{\mathbf{x}} = \overline{\mathbf{y}} \cdot \dfrac{\partial \mathbf{y}}{\partial \mathbf{x}} = \overline{\mathbf{y}} \cdot
\begin{bmatrix}
\dfrac{\partial y_1}{\partial x_1} & \dfrac{\partial y_1}{\partial x_2} & \dfrac{\partial y_1}{\partial x_3} \\
\dfrac{\partial y_2}{\partial x_1} & \dfrac{\partial y_2}{\partial x_2} & \dfrac{\partial y_2}{\partial x_3} \\
\dfrac{\partial y_3}{\partial x_1} & \dfrac{\partial y_3}{\partial x_2} & \dfrac{\partial y_3}{\partial x_3}
\end{bmatrix} = \begin{bmatrix} \overline{y}_1 & \overline{y}_2 & \overline{y}_3 \end{bmatrix} \cdot
\begin{bmatrix} 
2x_1 & 0 & 0 \\
0 & 2x_2 & 0 \\
0 & 0 & 2x_3
\end{bmatrix} = \begin{bmatrix} 2x_1 & 2x_2 &2 x_3 \end{bmatrix}$

In [9]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x**2
z = torch.sum(y)
print(f'x = {x.detach().numpy()}')
print(f'y = x^2 = {y.detach().numpy()}')
print(f'z = sum(y) = {z}')

x = [1. 2. 3.]
y = x^2 = [1. 4. 9.]
z = sum(y) = 14.0


In [10]:
z.backward()
print(f'dz/dx = {x.grad.detach().numpy()}')

dz/dx = [2. 4. 6.]


Có thể thấy khi thực hiện autodiff với node output là scalar, các node còn lại là các vector thường phải thực hiện phép nhân ma trận giữa vector gradient và ma trận Jacobian $\mathbf{v} \cdot \mathbf{J}$ (gọi là vector-Jacobian product hay vjp) $\rightarrow$ có cách nào để tính được vector-Jacobian product mà không cần thực hiện trực tiếp phép nhân ma trận?

Trở lại ví dụ ở trên 
$\overline{\mathbf{x}} = \overline{\mathbf{y}} \cdot \dfrac{\partial \mathbf{y}}{\partial \mathbf{x}} = 
\begin{bmatrix} \overline{y}_1 & \overline{y}_2 & \overline{y}_3 \end{bmatrix} \cdot
\begin{bmatrix} 
2x_1 & 0 & 0 \\
0 & 2x_2 & 0 \\
0 & 0 & 2x_3
\end{bmatrix} = 
\begin{bmatrix} \overline{y}_1 \cdot 2x_1 & \overline{y}_2 \cdot 2x_2 & \overline{y}_3 \cdot 2x_3 \end{bmatrix} = 2
\underbrace{\overline{\mathbf{y}} \odot \mathbf{x}}_{\text{element-wise product}}$

Như vậy có thể tính $\overline{\mathbf{x}}$ bằng cách nhân element-wise giữa $\overline{\mathbf{y}}$ và $\mathbf{x}$ (và nhân $2$ nữa) mà không cần phải trực tiếp thực hiện nhân ma trận.
\begin{align}
\text{vjp}_{(\cdot)^2}(\mathbf{v}, \mathbf{y}, \mathbf{x}) &= 2 \mathbf{v} \odot \mathbf{x} \\
\overline{\mathbf{x}} = \text{vjp}_{(\cdot)^2}(\overline{\mathbf{y}}, \mathbf{y}, \mathbf{x}) &= 2 \overline{\mathbf{y}} \odot \mathbf{x}
\end{align}

Ở ví dụ trên, nếu muốn tính $\text{vjp}_{(\cdot)^2}$ với vector $\mathbf{v}$ bất kì ta gọi `y.backward(gradient=v)` thay vì `z.backward()`

In [11]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x**2
print(f'x = {x.detach().numpy()}')
print(f'y = x^2 = {y.detach().numpy()}')

x = [1. 2. 3.]
y = x^2 = [1. 4. 9.]


In [12]:
v = torch.tensor([-1.0, 1.0, 2.0])
y.backward(gradient=v)
print(f'v = {v.numpy()}')
print(f'vjp_elementwise_square(v) = {x.grad.detach().numpy()}')

v = [-1.  1.  2.]
vjp_elementwise_square(v) = [-2.  4. 12.]


Tham khảo
* https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf
* https://github.com/mattjj/autodidact